<a href="https://colab.research.google.com/github/chipsalliance/silicon-notebooks/blob/main/xls-workshop-openlane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# From code to silicon using XLS and OpenLane

```
Copyright 2021 Google LLC.
SPDX-License-Identifier: Apache-2.0
```

- Design your hardware with code using [XLS](https://google.github.io/xls/) high-level synthesis toolkit.
- Run your design thru the [OpenLane](https://github.com/The-OpenROAD-Project/OpenLane/) RTL to GDS toolchain.
- Target the open source [SKY130](https://github.com/google/skywater-pdk/) PDK to manufacture your design using Google's sponsored [OpenMPW shuttle](https://developers.google.com/silicon) program.

In [ ]:
#@title Install dependencies {display-mode: "form"}
#@markdown - Click the ▷ button to setup the digital design environment based on [conda-eda](https://github.com/hdl/conda-eda).

openlane_version = 'latest' #@param {type:"string"}
open_pdks_version = 'latest' #@param {type:"string"}
xls_version = 'latest' #@param {type:"string"}

if openlane_version == 'latest':
  openlane_version = ''
if open_pdks_version == 'latest':
  open_pdks_version = ''
if xls_version == 'latest':
  xls_version = ''

import os
import pathlib
import sys

!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
site_package_path = conda_prefix_path / 'lib/python3.7/site-packages'
sys.path.append(str(site_package_path.resolve()))
CONDA_PREFIX = str(conda_prefix_path.resolve())
PATH = os.environ['PATH']
LD_LIBRARY_PATH = os.environ.get('LD_LIBRARY_PATH', '')
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}
%env LD_LIBRARY_PATH={CONDA_PREFIX}/lib:{LD_LIBRARY_PATH}
!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!CI=0 bin/micromamba install --yes --prefix $CONDA_PREFIX \
                     --channel litex-hub \
                     --channel main \
                     openlane={openlane_version} \
                     open_pdks.sky130a={open_pdks_version} \
                     xls={xls_version} \
                     iverilog
!python -m pip install cocotb pytest vcdvcd wavedrom
!curl -L -O https://patch-diff.githubusercontent.com/raw/The-OpenROAD-Project/OpenLane/pull/1503.patch
!patch -p1 -d conda-env/share/openlane < 1503.patch
!curl -L -O https://github.com/google/xls/archive/refs/heads/main.tar.gz
!tar --strip-components=1 -xf main.tar.gz xls-main/xls/dslx/stdlib/ xls-main/xls/modules/
def2gds_mag = '''gds read $::env(CONDA_PREFIX)/share/pdk/sky130A/libs.ref/sky130_fd_sc_hd/gds/sky130_fd_sc_hd.gds
lef read $::env(CONDA_PREFIX)/share/pdk/sky130A/libs.ref/sky130_fd_sc_hd/techlef/sky130_fd_sc_hd__nom.tlef
lef read $::env(CONDA_PREFIX)/share/pdk/sky130A/libs.ref/sky130_fd_sc_hd/lef/sky130_fd_sc_hd.lef
def read $::env(IN_DEF)
gds write $::env(IN_DEF).gds'''
with open('def2gds.mag', 'w') as f:
  f.write(def2gds_mag)
!git clone https://github.com/mbalestrini/GDS2glTF.git
!python -m pip install -r GDS2glTF/requirements.txt
!git clone https://github.com/proppy/gds_viewer.git
import jinja2
gds_viewer = jinja2.Environment(loader=jinja2.FileSystemLoader('gds_viewer')).get_template('viewer.html')

## Design with Code using HLS

[XLS](https://google.github.io/xls/) implements a High Level Synthesis (HLS) toolchain which enables the rapid development of hardware IP via "software style" methodology.

[DSLX](https://google.github.io/xls/dslx_reference/) is a domain specific, dataflow-oriented functional language used to build hardware from flexible, high-level descriptions of functionality.

![img](https://google.github.io/xls/images/xls_stack_diagram.png)

### DSLX

The following cell feature collection of DSLX functions demonstrating:
- Basic language constructs and [expressions](https://google.github.io/xls/dslx_reference/#expressions)
- [Unit testing](https://google.github.io/xls/dslx_reference/#unit-tests)
- [Parametrics functions](https://google.github.io/xls/dslx_reference/#parametric-functions)
- [Standard library](https://google.github.io/xls/dslx_std/) and module [imports](https://google.github.io/xls/dslx_reference/#imports).


In [ ]:
%%bash -c 'cat > user_module.x; interpreter_main user_module.x'

// The first line declares a function (fn) named "adder1" (Functions: https://google.github.io/xls/dslx_reference/#functions)
// This function accepts two 1-bit unsigned integers (u1) named "a" and "b", and returns a 2-bit unsigned integer (u2)
// Functions without a return value can also be defined by removing the "-> u2" part in this example

// The second line defines 2-bit unsigned integer (u2) named "n" and assigns the sum of "a" and "b"
// "let" statement introduces a new variable
// "as" converts the u1 type "a" variable to a u2 type variable (Type casting: https://google.github.io/xls/dslx_reference/#type-casting)

// The third line is the return value (Functions return the result of their last computed expression as their return value.)

fn adder1(a: u1, b: u1) -> u2 {
  let n: u2 = a as u2 + b as u2;
  n
}


// This is a function to turn the "adder1" function into 8-bit input, 8-bit output function
// Refer to "Bit Slice Expressions" section in DSLX reference (Bit Slice Expressions: https://google.github.io/xls/dslx_reference/#bit-slice-expressions)

fn user_module(io_in: u8) -> u8 {
  adder1(io_in[0:1], io_in[4:5]) as u8
}


// This is a test function that tests whether the functions work as intended
// Refer to "assert_eq, assert_lt" section in DSLX reference (assert_eq, assert_lt: https://google.github.io/xls/dslx_reference/#assert_eq-assert_lt)
// The underscore (_) in the "let _ = ..." part means that the returned values of the function is ignored
// The underscore (_) in the binary values ("0b0001_0001") is just inserted for readablility and can be ignored

#[test]
fn test() {
  let _ = assert_eq(adder1(u1:0b1, u1:0b1), u2:0b10);
  let _ = assert_eq(user_module(u8:0b0001_0001), u8:0b000000_10);
  _
}

## Convert to Hardware IR

The [XLS IR](https://google.github.io/xls/ir_semantics/) is a pure dataflow-oriented IR that has the static-single-assignment property, but is specialized for generating circuitry.

In [ ]:
!ir_converter_main --top=user_module user_module.x > user_module.ir
!opt_main user_module.ir > user_module_opt.ir
!cat user_module_opt.ir

## Generate RTL

XLS codegen outputs (System)Verilog [RTL](https://en.wikipedia.org/wiki/Register-transfer_level) for synthesis and simulation.

As a lowest common denominator, [Verilog](https://en.wikipedia.org/wiki/Verilog) output enables XLS generated designs to integrate into existing design flows.

In [ ]:
!codegen_main --use_system_verilog=false --module_name=user_module --generator=combinational user_module_opt.ir > user_module.v
!cat user_module.v

## Run the OpenLane flow

[OpenLane](https://openlane.readthedocs.io/en/latest/) is an automated [RTL](https://en.wikipedia.org/wiki/Register-transfer_level) to [GDSII](https://en.wikipedia.org/wiki/GDSII) flow based on several components including [OpenROAD](https://theopenroadproject.org/), [Yosys](https://yosyshq.net/yosys/), [Magic](http://www.opencircuitdesign.com/magic/), [Netgen](http://opencircuitdesign.com/netgen/) and custom methodology scripts for design exploration and optimization targeting [open source PDKs](https://github.com/google/open-source-pdks).

![img](https://openlane.readthedocs.io/en/latest/_images/flow_v1.png)

#### Configuration

[Documentation](https://openlane.readthedocs.io/en/latest/reference/configuration.html)

In [ ]:
%%writefile config.json
{
    "DESIGN_NAME": "user_module",
    "VERILOG_FILES": "dir::user_module.v",
    "CLOCK_TREE_SYNTH": false,
    "CLOCK_PERIOD": 10,
    "CLOCK_PORT": "clk",
    "CLOCK_NET": "ref::$CLOCK_PORT",
    "FP_SIZING": "absolute",
    "DIE_AREA": "0 0 50 50",
    "PL_TARGET_DENSITY": 0.30,
    "FP_PIN_ORDER_CFG": "dir::pin_order.cfg"
}

In [ ]:
%%writefile pin_order.cfg
#BUS_SORT

#W
io_in.*

#E
out.*

### Synthesis

- Input: [RTL](https://en.wikipedia.org/wiki/Register-transfer_level) (Verilog)
- Output: Technology mapped [netlist](https://en.wikipedia.org/wiki/Netlist) (Verilog)
- Metrics: Cell count and [timing closure](https://en.wikipedia.org/wiki/Timing_closure) estimate

[Documentation](https://openlane.readthedocs.io/en/latest/usage/hardening_macros.html#synthesis)

In [ ]:
%env PDK=sky130A
!flow.tcl -design . -to synthesis

In [ ]:
#@title Preview {display-mode: "form"}

import graphviz
import pathlib

dots = sorted(pathlib.Path('runs').glob('*/tmp/synthesis/post_techmap.dot'))
print(dots)
dot = graphviz.Source.from_file(dots[-1])
dot.engine = 'dot'
dot

### Floorplan

- Input: Technology mapped [netlist](https://en.wikipedia.org/wiki/Netlist) (Verilog)
- Output: Die Physical layout with PDN and I/O pins ([DEF](https://en.wikipedia.org/wiki/Design_Exchange_Format))
- Metrics: Area

[Documentation](https://openlane.readthedocs.io/en/latest/usage/hardening_macros.html#floorplan)

In [ ]:
%env PDK=sky130A
!flow.tcl -design . -to floorplan

In [ ]:
#@title Preview {display-mode: "form"}

import pathlib

STEP='floorplan'
in_def = sorted(pathlib.Path('runs').glob(f'*/results/{STEP}/*.def'))[-1].resolve()
!IN_DEF={in_def} magic -dnull -noconsole -rcfile {CONDA_PREFIX}/share/pdk/sky130A/libs.tech/magic/sky130A.magicrc < def2gds.mag
!python3 GDS2glTF/gds2gltf.py {in_def}.gds | tee gds2gltf.log
import IPython.display
IPython.display.clear_output(wait=True)
with open(f'{in_def}.gds.gltf') as f:
  gltf_data=f.read()
  output = gds_viewer.render(gltf_data=gltf_data)
IPython.display.HTML(output)

### Placement

- Input: Technology mapped [netlist](https://en.wikipedia.org/wiki/Netlist) (Verilog), Die Physical layout with PDN and I/O pins ([DEF](https://en.wikipedia.org/wiki/Design_Exchange_Format))
- Output: Physical layout with component cells placed ([DEF](https://en.wikipedia.org/wiki/Design_Exchange_Format))
- Metrics: Area, cell density, [timing closure](https://en.wikipedia.org/wiki/Timing_closure) estimate

[Documentation](https://openlane.readthedocs.io/en/latest/usage/hardening_macros.html#placement)

In [ ]:
%env PDK=sky130A
!flow.tcl -design . -to placement

In [ ]:
#@title Preview {display-mode: "form"}

import pathlib

STEP='placement'
in_def = sorted(pathlib.Path('runs').glob(f'*/results/{STEP}/*.def'))[-1].resolve()
!IN_DEF={in_def} magic -dnull -noconsole -rcfile {CONDA_PREFIX}/share/pdk/sky130A/libs.tech/magic/sky130A.magicrc < def2gds.mag
!python3 GDS2glTF/gds2gltf.py {in_def}.gds | tee gds2gltf.log
import IPython.display
IPython.display.clear_output(wait=True)
with open(f'{in_def}.gds.gltf') as f:
  gltf_data=f.read()
  output = gds_viewer.render(gltf_data=gltf_data)
IPython.display.HTML(output)

### Routing

- Input: Technology mapped [netlist](https://en.wikipedia.org/wiki/Netlist) (Verilog), Physical layout with component cells placed ([DEF](https://en.wikipedia.org/wiki/Design_Exchange_Format))
- Output: Physical layout with component cells fully-connected ([DEF](https://en.wikipedia.org/wiki/Design_Exchange_Format))
- Metrics: Routing congestion, [timing closure](https://en.wikipedia.org/wiki/Timing_closure) estimate

[Documentation](https://openlane.readthedocs.io/en/latest/usage/hardening_macros.html#routing)

In [ ]:
%env PDK=sky130A
!flow.tcl -design . -to routing

In [ ]:
#@title Preview {display-mode: "form"}

import pathlib

STEP='routing'
in_def = sorted(pathlib.Path('runs').glob(f'*/results/{STEP}/*.def'))[-1].resolve()
!IN_DEF={in_def} magic -dnull -noconsole -rcfile {CONDA_PREFIX}/share/pdk/sky130A/libs.tech/magic/sky130A.magicrc < def2gds.mag
!python3 GDS2glTF/gds2gltf.py {in_def}.gds | tee gds2gltf.log
import IPython.display
IPython.display.clear_output(wait=True)
with open(f'{in_def}.gds.gltf') as f:
  gltf_data=f.read()
  output = gds_viewer.render(gltf_data=gltf_data)
IPython.display.HTML(output)

### Sign off

- Input: Physical layout with component cells fully-connected ([DEF](https://en.wikipedia.org/wiki/Design_Exchange_Format))
- Output: Physical layout validated against foundry [DRC rules](https://en.wikipedia.org/wiki/Design_rule_checking), ready for manufacturing ([GDSII](https://en.wikipedia.org/wiki/GDSII))
- Metrics: DRC errors, [parasitics](https://en.wikipedia.org/wiki/Standard_Parasitic_Exchange_Format) and [timing closure](https://en.wikipedia.org/wiki/Timing_closure) estimate

[Documentation](https://openlane.readthedocs.io/en/latest/usage/hardening_macros.html#final-reports-and-checks)

In [ ]:
%env PDK=sky130A
!flow.tcl -design .

In [ ]:
#@title Preview {display-mode: "form"}

import pathlib

gds = sorted(pathlib.Path('runs').glob(f'*/results/final/gds/*.gds'))[-1].resolve()
!python3 GDS2glTF/gds2gltf.py {gds} | tee gds2gltf.log
import IPython.display
IPython.display.clear_output(wait=True)
with open(f'{gds}.gltf') as f:
  gltf_data=f.read()
  output = gds_viewer.render(gltf_data=gltf_data)
IPython.display.HTML(output)

In [ ]:
#@title Metrics {display-mode: "form"}
#@markdown [Documentation](https://openlane.readthedocs.io/en/latest/reference/datapoint_definitions.html)
import pathlib
import pandas as pd

pd.options.display.max_rows = None
csv = sorted(pathlib.Path('runs').glob('*/reports/metrics.csv'))
df = pd.read_csv(csv[-1])
df.transpose()

# Assignments

## Multiplier

### ⌨ DSLX

1. Update the `mul4` function below to use the [DSLX standard library](https://google.github.io/xls/dslx_std/) functions to implement a 4-bit multiplier (don't forget the `std::` prefix).
1. Generate the verilog for the design.
1. Run the OpenLane flow up until synthesis.
1. Observe the change in the complexity of the graph.
1. Compare to the results w/ the previous adder design.

In [ ]:
%%bash -c 'cat > user_module.x; interpreter_main user_module.x'
import std

fn mul4(a: u4, b: u4) -> u8 {
  u8:0 // TODO(YOU) implement mul4
}

fn user_module(io_in: u8) -> u8 {
  mul4(io_in[0:4], io_in[4:8]) as u8
}

#[test]
fn test() {
  let _ = assert_eq(mul4(u4:8, u4:8), u8:64);
  let _ = assert_eq(user_module(u8:0b1000_1000), u8:0b0100_0000);
  _
}

In [ ]:
!ir_converter_main --top=user_module user_module.x > user_module.ir
!opt_main user_module.ir > user_module_opt.ir
!codegen_main --use_system_verilog=false --module_name=user_module --generator=combinational user_module_opt.ir > user_module.v
!cat user_module.v

In [ ]:
%env PDK=sky130A
!flow.tcl -design . -to synthesis

In [ ]:
#@title Preview {display-mode: "form"}

import graphviz
import pathlib

dots = sorted(pathlib.Path('runs').glob('*/tmp/synthesis/post_techmap.dot'))
dot = graphviz.Source.from_file(dots[-1])
dot.engine = 'dot'
dot

### 🛠️ OpenLane

1. Run the complete OpenLane flow.
1. Fix errors by tweaking `DIE_AREA` or `PL_TARGET_DENSITY` [configuration variables](https://openlane.readthedocs.io/en/latest/reference/configuration.html).
1. Observe changes in the [metrics](https://openlane.readthedocs.io/en/latest/reference/datapoint_definitions.html) (`Total_Physical_Cells`, `wire_length`).
1. Observe changes in the layout preview.
1. Compare to the results w/ the previous adder design.

In [ ]:
%%writefile config.json
{
    "DESIGN_NAME": "user_module",
    "VERILOG_FILES": "dir::user_module.v",
    "CLOCK_TREE_SYNTH": false,
    "CLOCK_PERIOD": 10,
    "CLOCK_PORT": "clk",
    "CLOCK_NET": "ref::$CLOCK_PORT",
    "FP_SIZING": "absolute",
    "DIE_AREA": "0 0 50 50",
    "PL_TARGET_DENSITY": 0.30,
    "FP_PIN_ORDER_CFG": "dir::pin_order.cfg"
}

In [ ]:
%env PDK=sky130A
!flow.tcl -design .

In [ ]:
#@title Metrics {display-mode: "form"}
import pathlib
import pandas as pd

pd.options.display.max_rows = None
csv = sorted(pathlib.Path('runs').glob('*/reports/metrics.csv'))
df = pd.read_csv(csv[-1])
df.transpose()

In [ ]:
#@title Preview {display-mode: "form"}

import pathlib

gds = sorted(pathlib.Path('runs').glob(f'*/results/final/gds/*.gds'))[-1].resolve()
!python3 GDS2glTF/gds2gltf.py {gds} | tee gds2gltf.log
import IPython.display
IPython.display.clear_output(wait=True)
with open(f'{gds}.gltf') as f:
  gltf_data=f.read()
  output = gds_viewer.render(gltf_data=gltf_data)
IPython.display.HTML(output)

## Pipeline

### Pipeline stages

1. Find the right combination of XLS [codegen pipelining options](https://google.github.io/xls/codegen_options/#pipelining-and-scheduling-options) to [schedule](https://google.github.io/xls/scheduling/) the multiplier across multiple pipeline stages.
1. Observe the generated verilog

In [ ]:
pipeline_stages = 1
clock_period_ps = 10
!ir_converter_main --top=user_module user_module.x > user_module.ir
!opt_main user_module.ir > user_module_opt.ir
!codegen_main --pipeline_stages={pipeline_stages} --clock_period_ps={clock_period_ps} --delay_model=sky130 --use_system_verilog=false --module_name=user_module user_module_opt.ir > user_module.v
!cat user_module.v

### Simulation.

1. Update the [cocotb](https://www.cocotb.org/) test bench to verify the behavior of the generated verilog.
1. Observe the [digital timing diagram](https://en.wikipedia.org/wiki/Digital_timing_diagram) to see how values flow thru the pipeling stages

In [ ]:
%%writefile Makefile
SIM ?= icarus
TOPLEVEL_LANG ?= verilog
VERILOG_SOURCES += user_module.v
VERILOG_SOURCES += top.v
TOPLEVEL = top
MODULE = test

include $(shell cocotb-config --makefiles)/Makefile.sim

In [ ]:
%%writefile top.v

module top(
  input wire clk,
  input wire [7:0] io_in,
  output wire [7:0] out
);

  user_module user_module_tb(clk, io_in, out);
`ifdef COCOTB_SIM
initial begin
  $dumpfile ("user_module.vcd");
  $dumpvars (0, user_module_tb);
  #1;
end
`endif
endmodule

In [ ]:
%%bash -c "cat > test.py; make"
import cocotb
from cocotb.triggers import ClockCycles, RisingEdge
from cocotb.clock import Clock

@cocotb.test()
async def test_test(dut):
    c = Clock(dut.clk, 1, 'ns')
    cocotb.start_soon(c.start())

    # TODO(YOU): update with the 8-input to user_module
    dut.user_module_tb.io_in.value = NaN
    # pipeline stage 0: fetch
    await ClockCycles(dut.clk, 2)
    # TODO(YOU): verify that the input value propagate to stage #0
    assert dut.user_module_tb.p0_io_in == NaN

    # pipeline stage 1: slice
    await ClockCycles(dut.clk, 2)
    # TODO(YOU): update `p1_bit_slice_*` names to match the wire in the generated verilog
    # TODO(YOU): verify that the input value is sliced into 4-bit umul operands
    assert dut.user_module_tb.p1_bit_slice_TODO == NaN
    assert dut.user_module_tb.p1_bit_slice_TODO == NaN

    # pipeline stage 2: multiply
    await ClockCycles(dut.clk, 2)
    # TODO(YOU): verify that the output value match the multiplication result
    assert dut.user_module_tb.out == NaN

In [ ]:
#@title Preview {display-mode: "form"}

import wavedrom
import vcdvcd
import json

def filter_signals(top, signals, filter=[]):
  if len(filter) == 0:
    return signals
  for f in filter:
    for s in signals:
      if f in s and 'comb' not in s:
        yield s.replace(f'{top}.', '')

def signals_to_wave(filename, top, clk='clk', cycles=0, filter=[], period=500):
  vcd = vcdvcd.VCDVCD(filename)
  clk_tv = vcd[f'{top}.{clk}'].tv
  ticks = len(clk_tv) if cycles == 0 else cycles * 2
  yield {'name': clk, 'wave': 'P' + '.' * (cycles-1), 'period': 2 }
  for s in filter_signals(top, vcd.signals, filter):
    wave = ['x'] * ticks
    data = []
    for t, d in vcd[f'{top}.{s}'].tv:
      i = int(t/500)
      if d != 'x':
        wave[i] = '='
        data.append(hex(int(f'0b{d}', 2)))
      else:
        wave[i] = 'x'
    yield {'name': s, 'wave': ''.join(wave), 'data': ' '.join(data), 'phase': 0.5}

drom = {
    'signal': list(signals_to_wave('user_module.vcd', top='top.user_module_tb', cycles = pipeline_stages+1, signals = ['io_in[7:0]', 'p1_bit_slice_33[3:0]', 'p1_bit_slice_34[3:0]', 'out[7:0]']))
}
svg = wavedrom.render(json.dumps(drom))
display(svg)

### Static timing analysis

1. Run the OpenLane flow up until [Clock Tree Synthesis](https://en.wikipedia.org/wiki/Physical_design_(electronics)#Clock_tree_synthesis).
1. Fix errors by tweaking the `CLOCK_PERIOD` [configuration variable](https://openlane.readthedocs.io/en/latest/reference/configuration.html) until [static timing analysis](https://en.wikipedia.org/wiki/Static_timing_analysis) logs shows positive [slack](https://en.wikipedia.org/wiki/Static_timing_analysis#Definitions) `MET` (not `VIOLATED`) for all [corners](https://en.wikipedia.org/wiki/Static_timing_analysis#Corners_and_STA).
1. Re-harden the design.
1. Observe the impact of pipeling on the layout.

In [ ]:
%%writefile config.json
{
    "DESIGN_NAME": "user_module",
    "VERILOG_FILES": "dir::user_module.v",
    "CLOCK_TREE_SYNTH": true,
    "CLOCK_PERIOD": 1,
    "CLOCK_PORT": "clk",
    "CLOCK_NET": "ref::$CLOCK_PORT",
    "FP_SIZING": "absolute",
    "DIE_AREA": "0 0 150 150",
    "PL_TARGET_DENSITY": 0.70,
    "FP_PIN_ORDER_CFG": "dir::pin_order.cfg"
}

In [ ]:
%%writefile pin_order.cfg
#BUS_SORT

#W
io_in\[0\]
io_in\[1\]
io_in\[2\]
io_in\[3\]
clk
io_in\[4\]
io_in\[5\]
io_in\[6\]
io_in\[7\]

#E
out.*

In [ ]:
%env PDK=sky130A
!flow.tcl -design . -verbose 10 -to cts

In [ ]:
%env PDK=sky130A
!flow.tcl -design .

In [ ]:
#@title Metrics {display-mode: "form"}
import pathlib
import pandas as pd

pd.options.display.max_rows = None
csv = sorted(pathlib.Path('runs').glob('*/reports/metrics.csv'))
df = pd.read_csv(csv[-1])
df.transpose()

In [ ]:
#@title Preview {display-mode: "form"}

import pathlib

gds = sorted(pathlib.Path('runs').glob(f'*/results/final/gds/*.gds'))[-1].resolve()
!python3 GDS2glTF/gds2gltf.py {gds} | tee gds2gltf.log
import IPython.display
IPython.display.clear_output(wait=True)
with open(f'{gds}.gltf') as f:
  gltf_data=f.read()
  output = gds_viewer.render(gltf_data=gltf_data)
IPython.display.HTML(output)

## DIY Zone

1. Use the remaining to work on your own design.
1. Re-use functions from XLS [stdlib](https://github.com/google/xls/tree/main/xls/dslx/stdlib), [examples](https://github.com/google/xls/tree/main/xls/examples), [modules](https://github.com/google/xls/tree/main/xls/examples) or [third_party](https://github.com/google/xls/tree/main/third_party).

In [ ]:
%%bash -c 'cat > user_module.x; interpreter_main user_module.x'

fn user_module(io_in: u8) -> u8 {
  u8:0
}

#[test]
fn test() {
  let _ = assert_eq(user_module(u8:0), u8:0);
  _
}

In [ ]:
!ir_converter_main --top=user_module user_module.x > user_module.ir
!opt_main user_module.ir > user_module_opt.ir
!codegen_main --use_system_verilog=false --module_name=user_module --generator=combinational user_module_opt.ir > user_module.v
!cat user_module.v

In [ ]:
%env PDK=sky130A
!flow.tcl -design .

In [ ]:
#@title Metrics {display-mode: "form"}
import pathlib
import pandas as pd

pd.options.display.max_rows = None
csv = sorted(pathlib.Path('runs').glob('*/reports/metrics.csv'))
df = pd.read_csv(csv[-1])
df.transpose()

In [ ]:
#@title Preview {display-mode: "form"}

import pathlib

gds = sorted(pathlib.Path('runs').glob(f'*/results/final/gds/*.gds'))[-1].resolve()
!python3 GDS2glTF/gds2gltf.py {gds} | tee gds2gltf.log
import IPython.display
IPython.display.clear_output(wait=True)
with open(f'{gds}.gltf') as f:
  gltf_data=f.read()
  output = gds_viewer.render(gltf_data=gltf_data)
IPython.display.HTML(output)

# Tape-out

In order to onboard on the Google's sponsored [OpenMPW shuttle program](https://developers.google.com/silicon), your design needs to be integrated in the [Caravel SoC harness](https://caravel-harness.readthedocs.io/) using the [Caravel User Project](https://caravel-user-project.readthedocs.io/en/latest/) template and pass the [mpw pre-check](https://github.com/efabless/mpw_precheck) tests.

![caravel](https://caravel-user-project.readthedocs.io/en/latest/_static/layout.png)

- User project area: 2920 µm x 3520 µm
- SoC: RISC-V based ([specs](https://caravel-mgmt-soc-litex.readthedocs.io/en/latest/#features))
- I/O pins: 38 ([pinout](https://caravel-harness.readthedocs.io/en/latest/pinout.html))
- Communication: Wishbone bus and 128 logic analyser pins


## Submit your design with `tiny_user_project`

1. Create a new GitHub project using the [tiny_user_project template](https://github.com/proppy/tiny_user_project).
1. [Set GitHub Pages](https://tinytapeout.com/faq/#my-github-action-is-failing-on-the-pages-part) `Sources` as `GitHub Actions`
1. Update `verilog/rtl/user_module.v` with the generated verilog for your design.
1. Update `info.yaml` to match your project:
   - set `wokwi_id` to `0`
   - uncomment and set `source_files` to `- verilog/rtl/user_module.v`.
   - uncomment and set `top_module` to `user_module`.
   - update the `documentation` and the `inputs` / `outputs` mapping to match your projects.
1. Commit, push and check the `Actions` summary for the  ![user_project_ci](https://github.com/proppy/tiny_caravel_user_project/actions/workflows/user_project_ci.yml/badge.svg) workflow (if successful a new commit including the hardened files will be automatically created).
1. [Submit](https://platform.efabless.com/projects/create?project_definition=Open+MPW&shuttle=MPW-8) your project github repository to the next [Open MPW shuttle](https://platform.efabless.com/shuttles/MPW-8).